In [1]:
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from utils import ProgressBar,Dataset
import pickle
import random
plt.style.use('ggplot')
%matplotlib inline

In [2]:
en_sentences = []
zh_sentences = []

en_sentences_len = []
zh_sentences_len = []

pb = ProgressBar(worksum=10000000)
pb.startjob()
with open('middleresult/segmented_train_seg_by_word.txt') as fhdl:
    english_flag = False
    num = 0
    for line in fhdl:
        num += 1 if english_flag else 0
        linesp = line.strip()
        linesp = linesp.split(' ')
        english_flag = not english_flag 
        if english_flag:
            en_sentences.append(linesp)
            en_sentences_len.append(len(linesp))
        else:
            zh_sentences.append(linesp)
            zh_sentences_len.append(len(linesp))
        if english_flag and num % 10000 == 0:
            pb.complete(10000)

 100.00 % [==================================================>] 10000000/10000000 	 used:82s eta:0 s

In [3]:
en_words = list(map(lambda x:x.split('\t'), open('middleresult/en_vocab.txt').read().split('\n')))[:-1]
zh_words = list(map(lambda x:x.split('\t'), open('middleresult/zh_vocab.txt').read().split('\n')))[:-1]

# params

In [4]:
sentence_len = 50
english_words = 130000
chinese_words = 120000

# TOKENLIZER BY WORD
# OOV strategy : throw away

In [5]:
specialdic = {0:'<unk>',1:'<eos>',2:'<go>'}

en2index,index2en = {},{}
zh2index,index2zh = {},{}

for index,word in specialdic.items():
    en2index[word] = index
    zh2index[word] = index
    index2en[index] = word
    index2zh[index] = word
    
for index,ext in enumerate(en_words):
    (word,times) = ext
    index += len(specialdic)
    en2index[word] = index
    index2en[index] = word

for index,ext in enumerate(zh_words):
    (word,times) = ext
    index += len(specialdic)
    zh2index[word] = index
    index2zh[index] = word

In [31]:
en2index['<unk>']

0

In [6]:
train_x = []
pb = ProgressBar(worksum=len(en_sentences),info="processing english sentences")
pb.startjob()
for index,sentence in enumerate(en_sentences):
    train_x.append([en2index.get(word,en2index['<unk>']) for word in sentence])
    if index % 1000 == 0:
        pb.complete(1000)
    
print()
    
train_y = []
pb = ProgressBar(worksum=len(zh_sentences),info="processing chinese sentences")
pb.startjob()
for index,sentence in enumerate(zh_sentences):
    train_y.append([zh2index.get(word,zh2index['<unk>']) for word in sentence])
    if index % 1000 == 0:
        pb.complete(1000)
        

processing english sentences 100.00 % [==================================================>] 10000000/10000000 	 used:74s eta:0 s
processing chinese sentences 100.00 % [==================================================>] 10000000/10000000 	 used:62s eta:0 s

In [27]:
for round in range(5):
    index = random.choice(range(len(en_sentences)))
    print(' '.join([index2en.get(i) for i in train_x[index]]))
    print(' '.join([index2zh.get(i) for i in train_y[index]]) + "\n")

the education for higher vocational students majoring in engineering supervision mainly revolves around cultivating and improving " vocational ablity " .
高职 工程 监理 专业 学生 的 教育 主要 是 围绕 培养 和 提高 “ 职业 能力 ” 来 进行 。

okay , okay , i get it . what ?
好 吧 ， 是 的 ， 我 懂 了 。 什么 ？

well , that i have spent a great deal of time on .
不 我 花 了 很多 时间 的 那个 。

standard classification for serviceability of an office facility for manageability .
可管理性 的 办公设备 的 适用性 标准 分类 。

i ' m a button pusher , spam in a can .
我 就 会 按些 按钮 像 个 罐装 的 牛肉 而已 。



In [15]:
!mkdir data/preprocessing_tokenlizer

In [17]:
import pickle
with open('data/preprocessing_tokenlizer/sentence_tokened_by_word.pkl','wb') as whdl:
    pickle.dump((
         index2zh,
         zh2index,
         index2en,
         en2index,
         train_x,
         train_y,
    ),whdl)

In [18]:
! ls -l data/preprocessing_tokenlizer/sentence_tokened_by_word.pkl

-rw-rw-r-- 1 mtq mtq 755128925 Sep  8 17:38 data/preprocessing_tokenlizer/sentence_tokened_by_word.pkl
